In [1]:
from correctingagent.agents.PGMAgent import PGMCorrectingAgent, read_sentence
from correctingagent.agents.teacher import TeacherAgent, FaultyTeacherAgent
from correctingagent.world.world import RandomColoursWorld
from correctingagent.models.pgmmodels import InferenceType
from correctingagent.models.pgmmodels import *
from correctingagent.util.CPD_generation import variable_or_CPD

In [3]:
w = RandomColoursWorld(problem_directory="big_onerule_faulty/big_onerule_faulty0", 
                       problem_number=5, 
                       domain_file="blocks-domain-unstackable.pddl")
teacher = FaultyTeacherAgent()
agent = PGMCorrectingAgent(w, teacher=teacher, inference_type=InferenceType.SearchInference, 
                           max_beam_size=1000, p_direct=0.9)

Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain-unstackable.pddl
Parsing problem /home/mappelgren/Desktop/correcting-agent/data/big_onerule_faulty/big_onerule_faulty0/problem5.pddl
Using approximate search inference
p direct 0.9
p indirect 1
inference type InferenceType.SearchInference


In [4]:
plan = [('put', ['b9', 't0', 'tower0']), ('put', ['b8', 'b9', 'tower0']), 
        ('put', ['b7', 'b8', 'tower0']), ('put', ['b6', 'b7', 'tower0']), 
        ('put', ['b5', 'b6', 'tower0']), ('put', ['b4', 'b5', 'tower0']), 
        ('put', ['b3', 'b4', 'tower0']), ('put', ['b2', 'b3', 'tower0'])]

    
for action, args in plan:
    w.update(action, args)

print(action, args)
agent.get_correction("no, put green blocks on blue blocks", action, args)


put ['b2', 'b3', 'tower0']
fix bw? False
bw 0.5
fix bw? False
bw 0.5
table correction? False
{'V_1(all x.(green(x) -> exists y. (blue(y) & on(x,y))))': 0.16071428571428575, 'V_1(all y.(blue(y) -> exists x. (green(x) & on(x,y))))': 0.16071428571428575}
Is the top object green?
no
before clamp
0.0017857142857142854 {'V_1(all y.(blue(y) -> exists x. (green(x) & on(x,y))))': 0, 'green(b2)': 1, 'blue(b3)': 1, 'all y.(blue(y) -> exists x. (green(x) & on(x,y)))': 1, 'V_1(all x.(green(x) -> exists y. (blue(y) & on(x,y))))': 0, 'all x.(green(x) -> exists y. (blue(y) & on(x,y)))': 1, 'F(b2)': array([0.91453825, 0.13029081, 0.5900521 ]), 'F(b3)': array([0.03339435, 0.35577726, 0.94918457]), 'corr_1': 1}
0.016071428571428573 {'V_1(all y.(blue(y) -> exists x. (green(x) & on(x,y))))': 0, 'green(b2)': 1, 'blue(b3)': 0, 'all y.(blue(y) -> exists x. (green(x) & on(x,y)))': 1, 'V_1(all x.(green(x) -> exists y. (blue(y) & on(x,y))))': 0, 'all x.(green(x) -> exists y. (blue(y) & on(x,y)))': 0, 'F(b2)': ar

/home/mappelgren/Desktop/pgmpy/pgmpy/factors/discrete/DiscreteFactor.py:633: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi1.values = phi1.values[slice_]
/home/mappelgren/Desktop/pgmpy/pgmpy/factors/discrete/DiscreteFactor.py:621: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi.values = phi.values[slice_]


In [5]:
model = agent.pgm_model.ordered_models[0]

In [6]:
get_scope(model)

{'F(b2)',
 'F(b3)',
 'V_1(all x.(green(x) -> exists y. (blue(y) & on(x,y))))',
 'V_1(all y.(blue(y) -> exists x. (green(x) & on(x,y))))',
 'all x.(green(x) -> exists y. (blue(y) & on(x,y)))',
 'all y.(blue(y) -> exists x. (green(x) & on(x,y)))',
 'blue(b3)',
 'corr_1',
 'green(b2)'}

In [7]:
variable_or_CPD(2, p=0.9)

array([[0.9, 0.1, 0.1, 0.1],
       [0.1, 0.9, 0.9, 0.9]])

In [8]:
d = {'F(b2)': [0,0,0],
 'F(b3)': [0,0,0],
 'V_1(all x.(green(x) -> exists y. (blue(y) & on(x,y))))': 1,
 'V_1(all y.(blue(y) -> exists x. (green(x) & on(x,y))))': 0,
 'all x.(green(x) -> exists y. (blue(y) & on(x,y)))': 1,
 'all y.(blue(y) -> exists x. (green(x) & on(x,y)))': 0,
 'blue(b3)': 0,
 'corr_1': 1,
 'green(b2)': 1}

In [9]:
for factor in model.factors:
    print(factor, evaluate_factor(d, factor))

phi(V_1(all y.(blue(y) -> exists x. (green(x) & on(x,y)))):2, green(b2):2, blue(b3):2, all y.(blue(y) -> exists x. (green(x) & on(x,y))):2) 1.0
phi(all y.(blue(y) -> exists x. (green(x) & on(x,y))):2) 0.9
phi(corr_1:2, V_1(all x.(green(x) -> exists y. (blue(y) & on(x,y)))):2, V_1(all y.(blue(y) -> exists x. (green(x) & on(x,y)))):2) 0.9
<pgmpy.factors.continuous.ContinuousFactor.ContinuousFactor object at 0x7fc0b3207910> 0.5
phi(V_1(all x.(green(x) -> exists y. (blue(y) & on(x,y)))):2, green(b2):2, blue(b3):2, all x.(green(x) -> exists y. (blue(y) & on(x,y))):2) 1.0
<pgmpy.factors.continuous.ContinuousFactor.ContinuousFactor object at 0x7fc0b3207950> 0.5
phi(all x.(green(x) -> exists y. (blue(y) & on(x,y))):2) 0.1


In [10]:
evaluate_model(d, model)

0.020250000000000004

In [11]:
model.factors[4].values

array([[[[1., 1.],
         [1., 1.]],

        [[1., 0.],
         [1., 1.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 1.],
         [0., 0.]]]])

In [4]:
read_sentence("sorry, b8 and b0 are wrong because you must put green blocks on blue blocks")

Message(rel='on', o1=['green'], o2=['blue'], T='recover', o3=['b8', 'b0'])

In [27]:
agent.pgm_model.get_rule_probs()

{'all y.(blue(y) -> exists x. (b8(x) & on(x,y)))': 0.6666666666666666,
 'all x.(b8(x) -> exists y. (blue(y) & on(x,y)))': 0.6666666666666666}

In [26]:
agent.pgm_model.ordered_models

In [8]:
agent.update_goal()

In [17]:
print(teacher.correction(w, args))

In [11]:
agent.pgm_model.ordered_models[0].factors

[<DiscreteFactor representing phi(corr_1:2, V_1(all x.(green(x) -> exists y. (blue(y) & on(x,y)))):2, V_1(all y.(blue(y) -> exists x. (green(x) & on(x,y)))):2) at 0x7fd09a6186d0>,
 <DiscreteFactor representing phi(V_1(all y.(blue(y) -> exists x. (green(x) & on(x,y)))):2, green(b6):2, green(b7):2, green(b8):2, green(b9):2, blue(b7):2, blue(b8):2, blue(b9):2, green(b0):2, blue(b0):2, all y.(blue(y) -> exists x. (green(x) & on(x,y))):2) at 0x7fd09a6f1210>,
 <DiscreteFactor representing phi(all x.(green(x) -> exists y. (blue(y) & on(x,y))):2) at 0x7fd09a5f9110>,
 <DiscreteFactor representing phi(all y.(blue(y) -> exists x. (green(x) & on(x,y))):2) at 0x7fd09a5f9350>,
 <DiscreteFactor representing phi(V_1(all x.(green(x) -> exists y. (blue(y) & on(x,y)))):2, green(b6):2, green(b7):2, green(b8):2, green(b9):2, blue(b7):2, blue(b8):2, blue(b9):2, green(b0):2, blue(b0):2, all x.(green(x) -> exists y. (blue(y) & on(x,y))):2) at 0x7fd09a6f17d0>]

In [4]:
w.update("put", ['b0', 'b8'])
agent.no_correction("put", ['b0', 'b8'])

no correction ['b0', 'b8'] 2 [all y.(yellow(y) -> exists x. (red(x) & on(x,y))), all x.(red(x) -> exists y. (yellow(y) & on(x,y)))]


In [5]:
w.update("put", ['b7', 'b0'])
agent.get_correction("No, now you cannot put b2 in the tower because you must put orange blocks on purple blocks", "put", ['b7', 'b0'])

fix bw? False
bw 0.5
fix bw? False
bw 0.5
table correction? True
{'V_3(all x.(orange(x) -> exists y. (purple(y) & on(x,y))))': 0.5, 'V_3(all y.(purple(y) -> exists x. (orange(x) & on(x,y))))': 0.5}
Is the top object orange?
yes


In [8]:
agent.pgm_model.models[0].factors

[<DiscreteFactor representing phi(V_2(all y.(yellow(y) -> exists x. (red(x) & on(x,y)))):2, red(b0):2, yellow(b8):2, all y.(yellow(y) -> exists x. (red(x) & on(x,y))):2) at 0x7f74015a46d0>,
 <DiscreteFactor representing phi(V_2(all x.(red(x) -> exists y. (yellow(y) & on(x,y)))):2, red(b0):2, yellow(b8):2, all x.(red(x) -> exists y. (yellow(y) & on(x,y))):2) at 0x7f74015a4b90>,
 <DiscreteFactor representing phi(V_1(all y.(yellow(y) -> exists x. (red(x) & on(x,y)))):2, red(b7):2, yellow(b8):2, all y.(yellow(y) -> exists x. (red(x) & on(x,y))):2) at 0x7f74015f6910>,
 <DiscreteFactor representing phi(V_1(all x.(red(x) -> exists y. (yellow(y) & on(x,y)))):2, red(b7):2, yellow(b8):2, all x.(red(x) -> exists y. (yellow(y) & on(x,y))):2) at 0x7f74015f6fd0>,
 <DiscreteFactor representing phi(corr_2:2, V_2(all x.(red(x) -> exists y. (yellow(y) & on(x,y)))):2, V_2(all y.(yellow(y) -> exists x. (red(x) & on(x,y)))):2) at 0x7f740155e2d0>,
 <DiscreteFactor representing phi(all x.(red(x) -> exists y.

In [9]:
agent.pgm_model.models[1].factors

 <DiscreteFactor representing phi(V_3(all x.(orange(x) -> exists y. (purple(y) & on(x,y)))):2, orange(b7):2, purple(b0):2, orange(b2):2, purple(b2):2, all x.(orange(x) -> exists y. (purple(y) & on(x,y))):2) at 0x7f7401576910>,
 <DiscreteFactor representing phi(corr_3:2, V_3(all x.(orange(x) -> exists y. (purple(y) & on(x,y)))):2, V_3(all y.(purple(y) -> exists x. (orange(x) & on(x,y)))):2) at 0x7f7401567b90>,
 <DiscreteFactor representing phi(V_3(all y.(purple(y) -> exists x. (orange(x) & on(x,y)))):2, orange(b7):2, purple(b0):2, orange(b2):2, purple(b2):2, all y.(purple(y) -> exists x. (orange(x) & on(x,y))):2) at 0x7f74840dced0>,
 <DiscreteFactor representing phi(all x.(orange(x) -> exists y. (purple(y) & on(x,y))):2) at 0x7f7401576450>,
 <DiscreteFactor representing phi(all y.(purple(y) -> exists x. (orange(x) & on(x,y))):2) at 0x7f7401576550>]

In [7]:
for factor in agent.pgm_model.models[0].factors:
    print(factor.scope())

['F(b4)', 'purple(b4)']
['V_2(all y.(purple(y) -> exists x. (orange(x) & on(x,y))))', 'orange(b3)', 'purple(b4)', 'all y.(purple(y) -> exists x. (orange(x) & on(x,y)))']
['all x.(orange(x) -> exists y. (purple(y) & on(x,y)))']
['F(b3)', 'orange(b3)']
['F(b4)', 'purple(b4)']
['F(b9)', 'orange(b9)']
['V_2(all x.(orange(x) -> exists y. (purple(y) & on(x,y))))', 'orange(b3)', 'purple(b4)', 'all x.(orange(x) -> exists y. (purple(y) & on(x,y)))']
['all y.(purple(y) -> exists x. (orange(x) & on(x,y)))']
['corr_2', 'V_2(all x.(orange(x) -> exists y. (purple(y) & on(x,y))))', 'V_2(all y.(purple(y) -> exists x. (orange(x) & on(x,y))))']
['corr_1', 'V_1(all x.(orange(x) -> exists y. (purple(y) & on(x,y))))', 'V_1(all y.(purple(y) -> exists x. (orange(x) & on(x,y))))']
['V_1(all y.(purple(y) -> exists x. (orange(x) & on(x,y))))', 'orange(b9)', 'purple(b4)', 'all y.(purple(y) -> exists x. (orange(x) & on(x,y)))']
['V_1(all x.(orange(x) -> exists y. (purple(y) & on(x,y))))', 'orange(b9)', 'purple(b4

In [8]:
def is_equal(factor1, factor2):
    if type(factor1) != type(factor2):
        return False
    scope1 = factor1.scope()
    scope2 = factor2.scope()
    
    return len(scope1) == len(scope2) and all([n1 == n2 for n1, n2 in zip(scope1, scope2)])

In [9]:
factors = agent.pgm_model.models[0].factors
factor1 = factors[0]
factor2 = factors[1]
factor3 = factors[4]
print(is_equal(factor1, factor2))
print(is_equal(factor1, factor3))

False
True


In [11]:
any([])

False

In [3]:
w = RandomColoursWorld(problem_directory="big_tworules_random_colours/big_tworules_random_colours41", problem_number=1)
teacher = TeacherAgent()
agent = PGMCorrectingAgent(w, teacher=teacher, inference_type=InferenceType.BeliefPropagation)



Parsing domain /home/mappelgren/Desktop/correcting-agent/data/domain/blocks-domain.pddl
Parsing problem /home/mappelgren/Desktop/correcting-agent/data/big_tworules_random_colours/big_tworules_random_colours41/problem1.pddl


In [4]:
w.update("put", ['b9', 't0'])
agent.get_correction("no, put blue blocks on green blocks", "put", ['b9', 't0'])

fix bw? False
bw 0.5
fix bw? False
bw 0.5
table correction? False
{'V_1(all x.(blue(x) -> exists y. (green(y) & on(x,y))))': 1.0, 'V_1(all y.(green(y) -> exists x. (blue(x) & on(x,y))))': 0.0}


/home/mappelgren/Desktop/pgmpy/pgmpy/factors/discrete/DiscreteFactor.py:621: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi.values = phi.values[slice_]
/home/mappelgren/Desktop/pgmpy/pgmpy/factors/discrete/DiscreteFactor.py:633: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  phi1.values = phi1.values[slice_]
/home/mappelgren/Desktop/pgmpy/pgmpy/factors/discrete/DiscreteFactor.py:708: RuntimeWarning: invalid value encountered in true_divide
  phi.values = phi.values / phi1.values
/home/mappelgren/Desktop/pgmpy/pgmpy/factors/discrete/Disc